In [1]:
using DataFrames, CSV, StringEncodings, Dictionaries, Dates, StatsBase, HTTP, JSON, ProgressBars

In [2]:
datapath = joinpath("..", "Data")

"../Data"

## Read in the important raw files, we'll format this into a better formatted dataset after

In [3]:
yr = 2023

resultsdf = DataFrame(CSV.File(open(joinpath(datapath, "mlb-2025-asplayed.csv"), enc"ISO-8859-1")));
pitchingdf = DataFrame(CSV.File(joinpath(datapath, string(yr), "pitching_data_23.csv")));

In [4]:
# We need to reconcile the naming conventions - resultsdf has typed out teamnames while pitchingdf has shortforms and pks

teammap = DataFrame(CSV.File(joinpath(datapath, "TeamNameMap.csv")))
applymap = Dictionary(teammap.verbose, teammap.code)

resultsdf.HomeID = getindices(applymap, resultsdf.Home)
resultsdf.AwayID = getindices(applymap, resultsdf.Away);

In [4]:
pitchingdf.HomeID .= 0;
pitchingdf.AwayID .= 0;
pitchingdf.HomeRuns .= 0;
pitchingdf.AwayRuns .= 0;

println("There are " * string(size(pitchingdf, 1)/2) * " games")

There are 2430.0 games


In [7]:
subset = filter(row -> row.HomeID == 0, pitchingdf)

for pk = ProgressBar(unique(subset.game_pk))
    # Get the game 
    r = HTTP.request("GET", "https://baseballsavant.mlb.com/gf?game_pk=$pk")
    gameresult = JSON.parse(String(r.body))

    pitchingdf[pitchingdf.game_pk .== pk, :HomeID] .= gameresult["scoreboard"]["teams"]["home"]["id"]
    pitchingdf[pitchingdf.game_pk .== pk, :AwayID] .= gameresult["scoreboard"]["teams"]["away"]["id"]
    pitchingdf[pitchingdf.game_pk .== pk, :HomeRuns] .= gameresult["scoreboard"]["linescore"]["teams"]["home"]["runs"]
    pitchingdf[pitchingdf.game_pk .== pk, :AwayRuns] .= gameresult["scoreboard"]["linescore"]["teams"]["away"]["runs"]
end

0.0%┣                                              ┫ 0/597 [00:00<00:00, -0s/it]
0.2%┣                                          ┫ 1/597 [00:02<Inf:Inf, InfGs/it]
0.3%┣▏                                              ┫ 2/597 [00:04<39:13, 4s/it]
0.5%┣▎                                              ┫ 3/597 [00:07<33:10, 3s/it]
0.7%┣▎                                              ┫ 4/597 [00:09<29:03, 3s/it]
0.8%┣▍                                              ┫ 5/597 [00:12<30:06, 3s/it]
1.0%┣▌                                              ┫ 6/597 [00:16<31:50, 3s/it]
1.2%┣▌                                              ┫ 7/597 [00:18<30:13, 3s/it]
1.3%┣▋                                              ┫ 8/597 [00:22<30:20, 3s/it]
1.5%┣▊                                              ┫ 9/597 [00:26<31:25, 3s/it]
1.7%┣▊                                             ┫ 10/597 [00:28<30:37, 3s/it]
1.8%┣▉                                             ┫ 11/597 [00:31<30:06, 3s/it]
2.0%┣█                      

In [8]:
CSV.write("../Data/"*string(yr)*"/pitching_data_with_results.csv", pitchingdf)

"../Data/2023/pitching_data_with_results.csv"

In [5]:
# To match the game primary key let's setup a matching column that is of the format yymmddHHH where H is the
# home team identifier

tojoin = pitchingdf[:, [:player_id, :game_date, :game_pk]]
tojoin.joincol = string.(Dates.format.(tojoin.game_date, dateformat"yymmdd"), tojoin.player_id)

resultsdf.joincol = string.(Dates.format.(resultsdf.Date, dateformat"yymmdd"), resultsdf.HomeID)

dblheaders = [k for (k, v) in countmap(tojoin.joincol) if v > 1]

tojoin = filter(row -> !(row.joincol in dblheaders), tojoin)
tojoin = tojoin[:, [:game_pk, :joincol]]

resultsdf = leftjoin(resultsdf, tojoin, on = :joincol)

CSV.write("formanualdblheaders.csv", filter(row -> ismissing(row.game_pk), resultsdf))

"formanualdblheaders.csv"

In [ ]:
pk = tojoin[1, :game_pk]
println(pk)

r = HTTP.request("GET", "https://baseballsavant.mlb.com/gf?game_pk=$pk")
println(r.status)
gameresult = JSON.parse(String(r.body))
k = keys(gameresult)
println(gameresult["scoreboard"]["linescore"]["teams"])
print(keys(gameresult["scoreboard"]["teams"]["home"]))

778501200
Dict{String, Any}("home" => Dict{String, Any}("errors" => 1, "hits" => 10, "leftOnBase" => 8, "runs" => 3), "away" => Dict{String, Any}("errors" => 0, "hits" => 21, "leftOnBase" => 15, "runs" => 18))
["teamName", "active", "clubName", "link", "springVenue", "name", "id", "venue", "season", "firstYearOfPlay", "allStarStatus", "fileCode", "springLeague", "sport", "shortName", "franchiseName", "record", "abbreviation", "locationName", "league", "division", "teamCode"]